# 1. Tool to identify the electrical component events in CREAM

<p> This jupyter notebook was used to label the electrical appliance events in the data. <b> This is the first notebook in the labeling pipeline of CREAM. </b> </p>
<div class="alert alert-info">
    <h3>Instructions for labeling</h3>
    <p>Go through each day separately, by chaning the day of interest below. After you are done with a day, reset the corresponding cells. 
        <b>Please be aware, that every click into the figures gets recorded</b>. In case of errors (e.g. unintentional wrong click), you can execute the cell below each figure to remove the last click from the record </p>

<p> To save the events clicked, please use the corresponding cell below the figures.</p>

<p> <b> An event is defined as a 5 Ampere jump in the current signal. </b> </p>
<p> For turn-on events click at the foot of the event (before it increases) and for switch-off events select the peak of the event before the current drops </p> This helps to identify on- and off-events.
    
<p> <b> In case you are done with labeling everything (i.e. all days), you can uncomment and execute the last cell of the notebook to merge the individual files with the labels per hour into one overall file. </b> </p>
 
## Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import h5py
import pandas as pd
import scipy.interpolate
import os
import glob
import sys
from pathlib import Path
from datetime import datetime
from datetime import timedelta
import math

# Add project path to path for import
project_path = os.path.abspath("..")
if project_path not in sys.path:
    sys.path.append(project_path)

# Add module path to path for import
module_path = os.path.abspath("../data_utility/data_utility.py")
if module_path not in sys.path:
    sys.path.append(module_path)
    
from data_utility import CREAM_Day # class to work with a day of the CREAM Dataset

%matplotlib notebook
# DO NOT DELETE, needs to be loaded twice!
%matplotlib notebook  
%load_ext autoreload
%autoreload 2 # Reload all modules every time before executing the Python code typed.

## Global Settings

In [ ]:
SIGNAL_CHUNK_SIZE_MINUTES = 2
SAMPLING_FREQUENCY = 6400
FILE_LENGTH_MINUTES = 60
SIGNAL_CHUNK_SIZE_SAMPLES = SAMPLING_FREQUENCY * FILE_LENGTH_MINUTES  * SIGNAL_CHUNK_SIZE_MINUTES  # 2 minute chunks
TIME_PER_DATAPOINT = 1 / SAMPLING_FREQUENCY

## Global Function for deleting erros

In [ ]:
def delete_last_click(event_dictionary):
    """Deletes the last click from every key in the event_dictionary."""
    for k, v in event_dictionary.items():
        event_dictionary[k] = v[:-1]
    
    return event_dictionary


<div class="alert alert-danger">
    <h3>//TODO</h3>
    <p>Please specify the path to the main-folder of "CREAM". </p>
</div>

In [ ]:
PATH_TO_DATA = os.path.abspath(os.path.join("..", "..", "Datasets", "CREAM", "CREAM"))

<div class="alert alert-danger">
    <h3>//TODO</h3>
    <p>Please specify the path to the main folder where the labels should be saved. </p>
</div>

In [ ]:
PATH_TO_SAVE_LOCATION = os.path.abspath(os.path.join("..", "..", "Datasets", "CREAM", "tmp"))

# Start the Labelings

<div class="alert alert-danger">
    <h3>In case of labeling errros: </h3>
    <p>After each "hourly" cell, you can execute the code to delete last click. Just execute the cell</p>
</div>


<div class="alert alert-danger">
    <h3>//ToDo</h3>
    <p>After you are done with a day, select the new day by altering the <b> CURRENT_DAY </b> variable and reset the notebook </p>
</div>

In [ ]:
CURRENT_DAY = "2018-08-24"

In [ ]:
day_path = os.path.join(PATH_TO_DATA, CURRENT_DAY)
day_save_path = os.path.join(PATH_TO_SAVE_LOCATION, CURRENT_DAY)

if not os.path.isdir(day_save_path): # in case the folder to store the labels of the day has not been created yet
    os.mkdir(day_save_path)

current_CREAM_day = CREAM_Day(cream_day_location=day_path) 

files_of_day = current_CREAM_day.files  # all files from this day

event_dictionary = {
    "Filename": [],
    "Timestamp": [],
    "Amplitude": []
}

def onclick(event):
    """
    Function to be executed in case of a click event at a figure.
    """
    event_dictionary["Filename"].append(files_of_day[current_hour_index])
    event_dictionary["Timestamp"].append(math.floor(event.xdata))
    event_dictionary["Amplitude"].append(math.floor(event.ydata))

<div class="alert alert-danger">
    <p>In case the interactive mode of the figures is not working, reexecute the import cell. </p>
</div>

## 0. Hour  (6:00 - 7:00)

In [ ]:
%matplotlib notebook
# DO NOT DELETE, needs to be loaded twice!
%matplotlib notebook  

event_dictionary = {
    "Filename": [],
    "Timestamp": [],
    "Amplitude": []
}

current_hour_index = 0
current_day_path = os.path.join(day_path, files_of_day[current_hour_index])
voltage, current = current_CREAM_day.load_file(current_CREAM_day.files[current_hour_index], return_noise=False)
start_timestamp_file = current_CREAM_day.files_metadata_df.iloc[current_hour_index].Start_timestamp
end_timestamp_file = current_CREAM_day.files_metadata_df.iloc[current_hour_index].End_timestamp
whole_current_signal = current  # get the current signal of the whole file
number_of_chunks = int(len(whole_current_signal) / SIGNAL_CHUNK_SIZE_SAMPLES)
all_current_signal_chunks = np.array_split(whole_current_signal,number_of_chunks)

# plot 2 minute chunks
for chunk_number, current_signal_chunk in enumerate(all_current_signal_chunks):
    start_timestamp_chunk = start_timestamp_file + timedelta(minutes=int(SIGNAL_CHUNK_SIZE_MINUTES * chunk_number)) #compute the start of the chunk
    end_timestamp_chunk = start_timestamp_chunk + timedelta(minutes=SIGNAL_CHUNK_SIZE_MINUTES)
    
    fig, ax = plt.subplots(1,1)   
    fig.canvas.mpl_connect('button_press_event', onclick) #append event to figure
    
    xticks = np.arange(len(current_signal_chunk)) + (chunk_number * SIGNAL_CHUNK_SIZE_SAMPLES)
    ax.plot(xticks, current_signal_chunk, markersize=0.1) #plot, only hours
    
    ax.tick_params(axis='x', rotation=90) #rotate the xlabels
     
    if np.max(current_signal_chunk) < 1:
           ax.set_ylim([-6,6])
 
    plt.title(str(start_timestamp_chunk) + " - " + str(end_timestamp_chunk))
    
    plt.show()

<div class="alert alert-danger">
    <h3>Made an error?</h3>
    <p>Uncomment and execute the following cell to delete the last click.</p>
</div>

In [ ]:
# ONLY USE TO DELETE WRONG DATA - uncomment if needed
# event_dictionary = delete_last_click(event_dictionary)

<div class="alert alert-info">
    <h3>Save the labels</h3>
</div>

In [ ]:
save_path = os.path.join(day_save_path, str(str(current_CREAM_day.day_date) + "_hour-" + str(current_hour_index) + ".csv"))
event_df = pd.DataFrame(event_dictionary)
event_df["Timestamp"] = event_df["Timestamp"].apply(lambda index: start_timestamp_file + 
                                                    timedelta(seconds=(index * TIME_PER_DATAPOINT)))  
event_df.to_csv(save_path, sep=";", header=True, index=False, encoding="utf-8", decimal=".")


## 1. Hour  (7:00 - 8:00)

In [ ]:
%matplotlib notebook
# DO NOT DELETE, needs to be loaded twice!
%matplotlib notebook  

event_dictionary = {
    "Filename": [],
    "Timestamp": [],
    "Amplitude": []
}

current_hour_index = 1
current_day_path = os.path.join(day_path, files_of_day[current_hour_index])
voltage, current = current_CREAM_day.load_file(current_CREAM_day.files[current_hour_index], return_noise=False)
start_timestamp_file = current_CREAM_day.files_metadata_df.iloc[current_hour_index].Start_timestamp
end_timestamp_file = current_CREAM_day.files_metadata_df.iloc[current_hour_index].End_timestamp

whole_current_signal = current  # get the current signal of the whole file
number_of_chunks = int(len(whole_current_signal) / SIGNAL_CHUNK_SIZE_SAMPLES)
all_current_signal_chunks = np.array_split(whole_current_signal,number_of_chunks)

# plot 2 minute chunks
for chunk_number, current_signal_chunk in enumerate(all_current_signal_chunks):
    start_timestamp_chunk = start_timestamp_file + timedelta(minutes=int(SIGNAL_CHUNK_SIZE_MINUTES * chunk_number)) #compute the start of the chunk
    end_timestamp_chunk = start_timestamp_chunk + timedelta(minutes=SIGNAL_CHUNK_SIZE_MINUTES)
    
    fig, ax = plt.subplots(1,1)   
    fig.canvas.mpl_connect('button_press_event', onclick) #append event to figure
    
    xticks = np.arange(len(current_signal_chunk)) + (chunk_number * SIGNAL_CHUNK_SIZE_SAMPLES)
    ax.plot(xticks, current_signal_chunk, markersize=0.1) #plot, only hours
    
    ax.tick_params(axis='x', rotation=90) #rotate the xlabels
     
    if np.max(current_signal_chunk) < 1:
           ax.set_ylim([-6,6])
 
    plt.title(str(start_timestamp_chunk) + " - " + str(end_timestamp_chunk))
    
    plt.show()

<div class="alert alert-danger">
    <h3>Made an error?</h3>
    <p>Uncomment and execute the following cell to delete the last click.</p>
</div>

In [ ]:
# ONLY USE TO DELETE WRONG DATA - uncomment if needed
#event_dictionary = delete_last_click(event_dictionary)

<div class="alert alert-info">
    <h3>Save the labels</h3>
</div>

In [ ]:
save_path = os.path.join(day_save_path, str(str(current_CREAM_day.day_date) + "_hour-" + str(current_hour_index) + ".csv"))
event_df = pd.DataFrame(event_dictionary)
event_df["Timestamp"] = event_df["Timestamp"].apply(lambda index: start_timestamp_file + 
                                                    timedelta(seconds=(index * TIME_PER_DATAPOINT)))  
event_df.to_csv(save_path, sep=";", header=True, index=False, encoding="utf-8", decimal=".")


## 2. Hour (8:00 - 9:00)

In [ ]:
%matplotlib notebook
# DO NOT DELETE, needs to be loaded twice!
%matplotlib notebook  

event_dictionary = {
    "Filename": [],
    "Timestamp": [],
    "Amplitude": []
}

current_hour_index = 2
current_day_path = os.path.join(day_path, files_of_day[current_hour_index])
voltage, current = current_CREAM_day.load_file(current_CREAM_day.files[current_hour_index], return_noise=False)
start_timestamp_file = current_CREAM_day.files_metadata_df.iloc[current_hour_index].Start_timestamp
end_timestamp_file = current_CREAM_day.files_metadata_df.iloc[current_hour_index].End_timestamp

whole_current_signal = current  # get the current signal of the whole file
number_of_chunks = int(len(whole_current_signal) / SIGNAL_CHUNK_SIZE_SAMPLES)
all_current_signal_chunks = np.array_split(whole_current_signal,number_of_chunks)

# plot 2 minute chunks
for chunk_number, current_signal_chunk in enumerate(all_current_signal_chunks):
    start_timestamp_chunk = start_timestamp_file + timedelta(minutes=int(SIGNAL_CHUNK_SIZE_MINUTES * chunk_number)) #compute the start of the chunk
    end_timestamp_chunk = start_timestamp_chunk + timedelta(minutes=SIGNAL_CHUNK_SIZE_MINUTES)
    
    fig, ax = plt.subplots(1,1)   
    fig.canvas.mpl_connect('button_press_event', onclick) #append event to figure
    
    xticks = np.arange(len(current_signal_chunk)) + (chunk_number * SIGNAL_CHUNK_SIZE_SAMPLES)
    ax.plot(xticks, current_signal_chunk, markersize=0.1) #plot, only hours
    
    ax.tick_params(axis='x', rotation=90) #rotate the xlabels
     
    if np.max(current_signal_chunk) < 1:
           ax.set_ylim([-6,6])
 
    plt.title(str(start_timestamp_chunk) + " - " + str(end_timestamp_chunk))
    
    plt.show()

<div class="alert alert-danger">
    <h3>Made an error?</h3>
    <p>Uncomment and execute the following cell to delete the last click.</p>
</div>

In [ ]:
# ONLY USE TO DELETE WRONG DATA - uncomment if needed
#event_dictionary = delete_last_click(event_dictionary)

<div class="alert alert-info">
    <h3>Save the labels</h3>
</div>

In [ ]:
save_path = os.path.join(day_save_path, str(str(current_CREAM_day.day_date) + "_hour-" + str(current_hour_index) + ".csv"))
event_df = pd.DataFrame(event_dictionary)
event_df["Timestamp"] = event_df["Timestamp"].apply(lambda index: start_timestamp_file + 
                                                    timedelta(seconds=(index * TIME_PER_DATAPOINT)))  
event_df.to_csv(save_path, sep=";", header=True, index=False, encoding="utf-8", decimal=".")


## 3. Hour (9:00 - 10:00)

In [ ]:
%matplotlib notebook
# DO NOT DELETE, needs to be loaded twice!
%matplotlib notebook  

event_dictionary = {
    "Filename": [],
    "Timestamp": [],
    "Amplitude": []
}

current_hour_index = 3
current_day_path = os.path.join(day_path, files_of_day[current_hour_index])
voltage, current = current_CREAM_day.load_file(current_CREAM_day.files[current_hour_index], return_noise=False)
start_timestamp_file = current_CREAM_day.files_metadata_df.iloc[current_hour_index].Start_timestamp
end_timestamp_file = current_CREAM_day.files_metadata_df.iloc[current_hour_index].End_timestamp

whole_current_signal = current  # get the current signal of the whole file
number_of_chunks = int(len(whole_current_signal) / SIGNAL_CHUNK_SIZE_SAMPLES)
all_current_signal_chunks = np.array_split(whole_current_signal,number_of_chunks)

# plot 2 minute chunks
for chunk_number, current_signal_chunk in enumerate(all_current_signal_chunks):
    start_timestamp_chunk = start_timestamp_file + timedelta(minutes=int(SIGNAL_CHUNK_SIZE_MINUTES * chunk_number)) #compute the start of the chunk
    end_timestamp_chunk = start_timestamp_chunk + timedelta(minutes=SIGNAL_CHUNK_SIZE_MINUTES)
    
    fig, ax = plt.subplots(1,1)   
    fig.canvas.mpl_connect('button_press_event', onclick) #append event to figure
    
    xticks = np.arange(len(current_signal_chunk)) + (chunk_number * SIGNAL_CHUNK_SIZE_SAMPLES)
    ax.plot(xticks, current_signal_chunk, markersize=0.1) #plot, only hours
    
    ax.tick_params(axis='x', rotation=90) #rotate the xlabels
     
    if np.max(current_signal_chunk) < 1:
           ax.set_ylim([-6,6])
 
    plt.title(str(start_timestamp_chunk) + " - " + str(end_timestamp_chunk))
    
    plt.show()

<div class="alert alert-danger">
    <h3>Made an error?</h3>
    <p>Uncomment and execute the following cell to delete the last click.</p>
</div>

In [ ]:
# ONLY USE TO DELETE WRONG DATA - uncomment if needed
#event_dictionary = delete_last_click(event_dictionary)

<div class="alert alert-info">
    <h3>Save the labels</h3>
</div>

In [ ]:
save_path = os.path.join(day_save_path, str(str(current_CREAM_day.day_date) + "_hour-" + str(current_hour_index) + ".csv"))
event_df = pd.DataFrame(event_dictionary)
event_df["Timestamp"] = event_df["Timestamp"].apply(lambda index: start_timestamp_file + 
                                                    timedelta(seconds=(index * TIME_PER_DATAPOINT)))  
event_df.to_csv(save_path, sep=";", header=True, index=False, encoding="utf-8", decimal=".")


## 4. Hour (10:00 - 11:00)

In [ ]:
%matplotlib notebook
# DO NOT DELETE, needs to be loaded twice!
%matplotlib notebook  

event_dictionary = {
    "Filename": [],
    "Timestamp": [],
    "Amplitude": []
}

current_hour_index = 4
current_day_path = os.path.join(day_path, files_of_day[current_hour_index])
voltage, current = current_CREAM_day.load_file(current_CREAM_day.files[current_hour_index], return_noise=False)
start_timestamp_file = current_CREAM_day.files_metadata_df.iloc[current_hour_index].Start_timestamp
end_timestamp_file = current_CREAM_day.files_metadata_df.iloc[current_hour_index].End_timestamp

whole_current_signal = current  # get the current signal of the whole file
number_of_chunks = int(len(whole_current_signal) / SIGNAL_CHUNK_SIZE_SAMPLES)
all_current_signal_chunks = np.array_split(whole_current_signal,number_of_chunks)

# plot 2 minute chunks
for chunk_number, current_signal_chunk in enumerate(all_current_signal_chunks):
    start_timestamp_chunk = start_timestamp_file + timedelta(minutes=int(SIGNAL_CHUNK_SIZE_MINUTES * chunk_number)) #compute the start of the chunk
    end_timestamp_chunk = start_timestamp_chunk + timedelta(minutes=SIGNAL_CHUNK_SIZE_MINUTES)
    
    fig, ax = plt.subplots(1,1)   
    fig.canvas.mpl_connect('button_press_event', onclick) #append event to figure
    
    xticks = np.arange(len(current_signal_chunk)) + (chunk_number * SIGNAL_CHUNK_SIZE_SAMPLES)
    ax.plot(xticks, current_signal_chunk, markersize=0.1) #plot, only hours
    
    ax.tick_params(axis='x', rotation=90) #rotate the xlabels
     
    if np.max(current_signal_chunk) < 1:
           ax.set_ylim([-6,6])
 
    plt.title(str(start_timestamp_chunk) + " - " + str(end_timestamp_chunk))
    
    plt.show()

<div class="alert alert-danger">
    <h3>Made an error?</h3>
    <p>Uncomment and execute the following cell to delete the last click.</p>
</div>

In [ ]:
# ONLY USE TO DELETE WRONG DATA - uncomment if needed
#event_dictionary = delete_last_click(event_dictionary)

<div class="alert alert-info">
    <h3>Save the labels</h3>
</div>

In [ ]:
save_path = os.path.join(day_save_path, str(str(current_CREAM_day.day_date) + "_hour-" + str(current_hour_index) + ".csv"))
event_df = pd.DataFrame(event_dictionary)
event_df["Timestamp"] = event_df["Timestamp"].apply(lambda index: start_timestamp_file + 
                                                    timedelta(seconds=(index * TIME_PER_DATAPOINT)))  
event_df.to_csv(save_path, sep=";", header=True, index=False, encoding="utf-8", decimal=".")


## 5. Hour (11:00 - 12:00)

In [ ]:
%matplotlib notebook
# DO NOT DELETE, needs to be loaded twice!
%matplotlib notebook  

event_dictionary = {
    "Filename": [],
    "Timestamp": [],
    "Amplitude": []
}

current_hour_index = 5
current_day_path = os.path.join(day_path, files_of_day[current_hour_index])
voltage, current = current_CREAM_day.load_file(current_CREAM_day.files[current_hour_index], return_noise=False)
start_timestamp_file = current_CREAM_day.files_metadata_df.iloc[current_hour_index].Start_timestamp
end_timestamp_file = current_CREAM_day.files_metadata_df.iloc[current_hour_index].End_timestamp

whole_current_signal = current  # get the current signal of the whole file
number_of_chunks = int(len(whole_current_signal) / SIGNAL_CHUNK_SIZE_SAMPLES)
all_current_signal_chunks = np.array_split(whole_current_signal,number_of_chunks)

# plot 2 minute chunks
for chunk_number, current_signal_chunk in enumerate(all_current_signal_chunks):
    start_timestamp_chunk = start_timestamp_file + timedelta(minutes=int(SIGNAL_CHUNK_SIZE_MINUTES * chunk_number)) #compute the start of the chunk
    end_timestamp_chunk = start_timestamp_chunk + timedelta(minutes=SIGNAL_CHUNK_SIZE_MINUTES)
    
    fig, ax = plt.subplots(1,1)   
    fig.canvas.mpl_connect('button_press_event', onclick) #append event to figure
    
    xticks = np.arange(len(current_signal_chunk)) + (chunk_number * SIGNAL_CHUNK_SIZE_SAMPLES)
    ax.plot(xticks, current_signal_chunk, markersize=0.1) #plot, only hours
    
    ax.tick_params(axis='x', rotation=90) #rotate the xlabels
     
    if np.max(current_signal_chunk) < 1:
           ax.set_ylim([-6,6])
 
    plt.title(str(start_timestamp_chunk) + " - " + str(end_timestamp_chunk))
    
    plt.show()

<div class="alert alert-danger">
    <h3>Made an error?</h3>
    <p>Uncomment and execute the following cell to delete the last click.</p>
</div>

In [ ]:
# ONLY USE TO DELETE WRONG DATA - uncomment if needed
#event_dictionary = delete_last_click(event_dictionary)

<div class="alert alert-info">
    <h3>Save the labels</h3>
</div>

In [ ]:
save_path = os.path.join(day_save_path, str(str(current_CREAM_day.day_date) + "_hour-" + str(current_hour_index) + ".csv"))
event_df = pd.DataFrame(event_dictionary)
event_df["Timestamp"] = event_df["Timestamp"].apply(lambda index: start_timestamp_file + 
                                                    timedelta(seconds=(index * TIME_PER_DATAPOINT)))  
event_df.to_csv(save_path, sep=";", header=True, index=False, encoding="utf-8", decimal=".")


## 6. Hour (12:00 - 13:00)

In [ ]:
%matplotlib notebook
# DO NOT DELETE, needs to be loaded twice!
%matplotlib notebook  

event_dictionary = {
    "Filename": [],
    "Timestamp": [],
    "Amplitude": []
}

current_hour_index = 6
current_day_path = os.path.join(day_path, files_of_day[current_hour_index])
voltage, current = current_CREAM_day.load_file(current_CREAM_day.files[current_hour_index], return_noise=False)
start_timestamp_file = current_CREAM_day.files_metadata_df.iloc[current_hour_index].Start_timestamp
end_timestamp_file = current_CREAM_day.files_metadata_df.iloc[current_hour_index].End_timestamp

whole_current_signal = current  # get the current signal of the whole file
number_of_chunks = int(len(whole_current_signal) / SIGNAL_CHUNK_SIZE_SAMPLES)
all_current_signal_chunks = np.array_split(whole_current_signal,number_of_chunks)

# plot 2 minute chunks
for chunk_number, current_signal_chunk in enumerate(all_current_signal_chunks):
    start_timestamp_chunk = start_timestamp_file + timedelta(minutes=int(SIGNAL_CHUNK_SIZE_MINUTES * chunk_number)) #compute the start of the chunk
    end_timestamp_chunk = start_timestamp_chunk + timedelta(minutes=SIGNAL_CHUNK_SIZE_MINUTES)
    
    fig, ax = plt.subplots(1,1)   
    fig.canvas.mpl_connect('button_press_event', onclick) #append event to figure
    
    xticks = np.arange(len(current_signal_chunk)) + (chunk_number * SIGNAL_CHUNK_SIZE_SAMPLES)
    ax.plot(xticks, current_signal_chunk, markersize=0.1) #plot, only hours
    
    ax.tick_params(axis='x', rotation=90) #rotate the xlabels
     
    if np.max(current_signal_chunk) < 1:
           ax.set_ylim([-6,6])
 
    plt.title(str(start_timestamp_chunk) + " - " + str(end_timestamp_chunk))
    
    plt.show()

<div class="alert alert-danger">
    <h3>Made an error?</h3>
    <p>Uncomment and execute the following cell to delete the last click.</p>
</div>

In [ ]:
# ONLY USE TO DELETE WRONG DATA - uncomment if needed
#event_dictionary = delete_last_click(event_dictionary)

<div class="alert alert-info">
    <h3>Save the labels</h3>
</div>

In [ ]:
save_path = os.path.join(day_save_path, str(str(current_CREAM_day.day_date) + "_hour-" + str(current_hour_index) + ".csv"))
event_df = pd.DataFrame(event_dictionary)
event_df["Timestamp"] = event_df["Timestamp"].apply(lambda index: start_timestamp_file + 
                                                    timedelta(seconds=(index * TIME_PER_DATAPOINT)))  
event_df.to_csv(save_path, sep=";", header=True, index=False, encoding="utf-8", decimal=".")


## 7. Hour (13:00 - 14:00)

In [ ]:
%matplotlib notebook
# DO NOT DELETE, needs to be loaded twice!
%matplotlib notebook  

event_dictionary = {
    "Filename": [],
    "Timestamp": [],
    "Amplitude": []
}

current_hour_index = 7
current_day_path = os.path.join(day_path, files_of_day[current_hour_index])
voltage, current = current_CREAM_day.load_file(current_CREAM_day.files[current_hour_index], return_noise=False)
start_timestamp_file = current_CREAM_day.files_metadata_df.iloc[current_hour_index].Start_timestamp
end_timestamp_file = current_CREAM_day.files_metadata_df.iloc[current_hour_index].End_timestamp

whole_current_signal = current  # get the current signal of the whole file
number_of_chunks = int(len(whole_current_signal) / SIGNAL_CHUNK_SIZE_SAMPLES)
all_current_signal_chunks = np.array_split(whole_current_signal,number_of_chunks)

# plot 2 minute chunks
for chunk_number, current_signal_chunk in enumerate(all_current_signal_chunks):
    start_timestamp_chunk = start_timestamp_file + timedelta(minutes=int(SIGNAL_CHUNK_SIZE_MINUTES * chunk_number)) #compute the start of the chunk
    end_timestamp_chunk = start_timestamp_chunk + timedelta(minutes=SIGNAL_CHUNK_SIZE_MINUTES)
    
    fig, ax = plt.subplots(1,1)   
    fig.canvas.mpl_connect('button_press_event', onclick) #append event to figure
    
    xticks = np.arange(len(current_signal_chunk)) + (chunk_number * SIGNAL_CHUNK_SIZE_SAMPLES)
    ax.plot(xticks, current_signal_chunk, markersize=0.1) #plot, only hours
    
    ax.tick_params(axis='x', rotation=90) #rotate the xlabels
     
    if np.max(current_signal_chunk) < 1:
           ax.set_ylim([-6,6])
 
    plt.title(str(start_timestamp_chunk) + " - " + str(end_timestamp_chunk))
    
    plt.show()

<div class="alert alert-danger">
    <h3>Made an error?</h3>
    <p>Uncomment and execute the following cell to delete the last click.</p>
</div>

In [ ]:
# ONLY USE TO DELETE WRONG DATA - uncomment if needed
#event_dictionary = delete_last_click(event_dictionary)

<div class="alert alert-info">
    <h3>Save the labels</h3>
</div>

In [ ]:
save_path = os.path.join(day_save_path, str(str(current_CREAM_day.day_date) + "_hour-" + str(current_hour_index) + ".csv"))
event_df = pd.DataFrame(event_dictionary)
event_df["Timestamp"] = event_df["Timestamp"].apply(lambda index: start_timestamp_file + 
                                                    timedelta(seconds=(index * TIME_PER_DATAPOINT)))  
event_df.to_csv(save_path, sep=";", header=True, index=False, encoding="utf-8", decimal=".")


## 8. Hour (14:00 - 15:00)

In [ ]:
%matplotlib notebook
# DO NOT DELETE, needs to be loaded twice!
%matplotlib notebook  

event_dictionary = {
    "Filename": [],
    "Timestamp": [],
    "Amplitude": []
}

current_hour_index = 8
current_day_path = os.path.join(day_path, files_of_day[current_hour_index])
voltage, current = current_CREAM_day.load_file(current_CREAM_day.files[current_hour_index], return_noise=False)
start_timestamp_file = current_CREAM_day.files_metadata_df.iloc[current_hour_index].Start_timestamp
end_timestamp_file = current_CREAM_day.files_metadata_df.iloc[current_hour_index].End_timestamp

whole_current_signal = current  # get the current signal of the whole file
number_of_chunks = int(len(whole_current_signal) / SIGNAL_CHUNK_SIZE_SAMPLES)
all_current_signal_chunks = np.array_split(whole_current_signal,number_of_chunks)

# plot 2 minute chunks
for chunk_number, current_signal_chunk in enumerate(all_current_signal_chunks):
    start_timestamp_chunk = start_timestamp_file + timedelta(minutes=int(SIGNAL_CHUNK_SIZE_MINUTES * chunk_number)) #compute the start of the chunk
    end_timestamp_chunk = start_timestamp_chunk + timedelta(minutes=SIGNAL_CHUNK_SIZE_MINUTES)
    
    fig, ax = plt.subplots(1,1)   
    fig.canvas.mpl_connect('button_press_event', onclick) #append event to figure
    
    xticks = np.arange(len(current_signal_chunk)) + (chunk_number * SIGNAL_CHUNK_SIZE_SAMPLES)
    ax.plot(xticks, current_signal_chunk, markersize=0.1) #plot, only hours
    
    ax.tick_params(axis='x', rotation=90) #rotate the xlabels
     
    if np.max(current_signal_chunk) < 1:
           ax.set_ylim([-6,6])
 
    plt.title(str(start_timestamp_chunk) + " - " + str(end_timestamp_chunk))
    
    plt.show()

<div class="alert alert-danger">
    <h3>Made an error?</h3>
    <p>Uncomment and execute the following cell to delete the last click.</p>
</div>

In [ ]:
# ONLY USE TO DELETE WRONG DATA - uncomment if needed
#event_dictionary = delete_last_click(event_dictionary)

<div class="alert alert-info">
    <h3>Save the labels</h3>
</div>

In [ ]:
save_path = os.path.join(day_save_path, str(str(current_CREAM_day.day_date) + "_hour-" + str(current_hour_index) + ".csv"))
event_df = pd.DataFrame(event_dictionary)
event_df["Timestamp"] = event_df["Timestamp"].apply(lambda index: start_timestamp_file + 
                                                    timedelta(seconds=(index * TIME_PER_DATAPOINT)))  
event_df.to_csv(save_path, sep=";", header=True, index=False, encoding="utf-8", decimal=".")


## 9. Hour (15:00 - 16:00)

In [ ]:
%matplotlib notebook
# DO NOT DELETE, needs to be loaded twice!
%matplotlib notebook  

event_dictionary = {
    "Filename": [],
    "Timestamp": [],
    "Amplitude": []
}

current_hour_index = 9
current_day_path = os.path.join(day_path, files_of_day[current_hour_index])
voltage, current = current_CREAM_day.load_file(current_CREAM_day.files[current_hour_index], return_noise=False)
start_timestamp_file = current_CREAM_day.files_metadata_df.iloc[current_hour_index].Start_timestamp
end_timestamp_file = current_CREAM_day.files_metadata_df.iloc[current_hour_index].End_timestamp

whole_current_signal = current  # get the current signal of the whole file
number_of_chunks = int(len(whole_current_signal) / SIGNAL_CHUNK_SIZE_SAMPLES)
all_current_signal_chunks = np.array_split(whole_current_signal,number_of_chunks)

# plot 2 minute chunks
for chunk_number, current_signal_chunk in enumerate(all_current_signal_chunks):
    start_timestamp_chunk = start_timestamp_file + timedelta(minutes=int(SIGNAL_CHUNK_SIZE_MINUTES * chunk_number)) #compute the start of the chunk
    end_timestamp_chunk = start_timestamp_chunk + timedelta(minutes=SIGNAL_CHUNK_SIZE_MINUTES)
    
    fig, ax = plt.subplots(1,1)   
    fig.canvas.mpl_connect('button_press_event', onclick) #append event to figure
    
    xticks = np.arange(len(current_signal_chunk)) + (chunk_number * SIGNAL_CHUNK_SIZE_SAMPLES)
    ax.plot(xticks, current_signal_chunk, markersize=0.1) #plot, only hours
    
    ax.tick_params(axis='x', rotation=90) #rotate the xlabels
     
    if np.max(current_signal_chunk) < 1:
           ax.set_ylim([-6,6])
 
    plt.title(str(start_timestamp_chunk) + " - " + str(end_timestamp_chunk))
    
    plt.show()

<div class="alert alert-danger">
    <h3>Made an error?</h3>
    <p>Uncomment and execute the following cell to delete the last click.</p>
</div>

In [ ]:
# ONLY USE TO DELETE WRONG DATA - uncomment if needed
#event_dictionary = delete_last_click(event_dictionary)

<div class="alert alert-info">
    <h3>Save the labels</h3>
</div>

In [ ]:
save_path = os.path.join(day_save_path, str(str(current_CREAM_day.day_date) + "_hour-" + str(current_hour_index) + ".csv"))
event_df = pd.DataFrame(event_dictionary)
event_df["Timestamp"] = event_df["Timestamp"].apply(lambda index: start_timestamp_file + 
                                                    timedelta(seconds=(index * TIME_PER_DATAPOINT)))  
event_df.to_csv(save_path, sep=";", header=True, index=False, encoding="utf-8", decimal=".")


## 10. Hour (16:00 - 17:00)

In [ ]:
%matplotlib notebook
# DO NOT DELETE, needs to be loaded twice!
%matplotlib notebook  

event_dictionary = {
    "Filename": [],
    "Timestamp": [],
    "Amplitude": []
}

current_hour_index = 10
current_day_path = os.path.join(day_path, files_of_day[current_hour_index])
voltage, current = current_CREAM_day.load_file(current_CREAM_day.files[current_hour_index], return_noise=False)
start_timestamp_file = current_CREAM_day.files_metadata_df.iloc[current_hour_index].Start_timestamp
end_timestamp_file = current_CREAM_day.files_metadata_df.iloc[current_hour_index].End_timestamp

whole_current_signal = current # get the current signal of the whole file
number_of_chunks = int(len(whole_current_signal) / SIGNAL_CHUNK_SIZE_SAMPLES)
all_current_signal_chunks = np.array_split(whole_current_signal,number_of_chunks)

# plot 2 minute chunks
for chunk_number, current_signal_chunk in enumerate(all_current_signal_chunks):
    start_timestamp_chunk = start_timestamp_file + timedelta(minutes=int(SIGNAL_CHUNK_SIZE_MINUTES * chunk_number)) #compute the start of the chunk
    end_timestamp_chunk = start_timestamp_chunk + timedelta(minutes=SIGNAL_CHUNK_SIZE_MINUTES)
    
    fig, ax = plt.subplots(1,1)   
    fig.canvas.mpl_connect('button_press_event', onclick) #append event to figure
    
    xticks = np.arange(len(current_signal_chunk)) + (chunk_number * SIGNAL_CHUNK_SIZE_SAMPLES)
    ax.plot(xticks, current_signal_chunk, markersize=0.1) #plot, only hours
    
    ax.tick_params(axis='x', rotation=90) #rotate the xlabels
     
    if np.max(current_signal_chunk) < 1:
           ax.set_ylim([-6,6])
 
    plt.title(str(start_timestamp_chunk) + " - " + str(end_timestamp_chunk))
    
    plt.show()

<div class="alert alert-danger">
    <h3>Made an error?</h3>
    <p>Uncomment and execute the following cell to delete the last click.</p>
</div>

In [ ]:
# ONLY USE TO DELETE WRONG DATA - uncomment if needed
#event_dictionary = delete_last_click(event_dictionary)

<div class="alert alert-info">
    <h3>Save the labels</h3>
</div>

In [ ]:
save_path = os.path.join(day_save_path, str(str(current_CREAM_day.day_date) + "_hour-" + str(current_hour_index) + ".csv"))
event_df = pd.DataFrame(event_dictionary)
event_df["Timestamp"] = event_df["Timestamp"].apply(lambda index: start_timestamp_file + 
                                                    timedelta(seconds=(index * TIME_PER_DATAPOINT)))  
event_df.to_csv(save_path, sep=";", header=True, index=False, encoding="utf-8", decimal=".")


## 11. Hour (18:00 - 19:00)

In [ ]:
%matplotlib notebook
# DO NOT DELETE, needs to be loaded twice!
%matplotlib notebook  

event_dictionary = {
    "Filename": [],
    "Timestamp": [],
    "Amplitude": []
}

current_hour_index = 11
current_day_path = os.path.join(day_path, files_of_day[current_hour_index])
voltage, current = current_CREAM_day.load_file(current_CREAM_day.files[current_hour_index], return_noise=False)
start_timestamp_file = current_CREAM_day.files_metadata_df.iloc[current_hour_index].Start_timestamp
end_timestamp_file = current_CREAM_day.files_metadata_df.iloc[current_hour_index].End_timestamp

whole_current_signal = current  # get the current signal of the whole file
number_of_chunks = int(len(whole_current_signal) / SIGNAL_CHUNK_SIZE_SAMPLES)
all_current_signal_chunks = np.array_split(whole_current_signal,number_of_chunks)

# plot 2 minute chunks
for chunk_number, current_signal_chunk in enumerate(all_current_signal_chunks):
    start_timestamp_chunk = start_timestamp_file + timedelta(minutes=int(SIGNAL_CHUNK_SIZE_MINUTES * chunk_number)) #compute the start of the chunk
    end_timestamp_chunk = start_timestamp_chunk + timedelta(minutes=SIGNAL_CHUNK_SIZE_MINUTES)
    
    fig, ax = plt.subplots(1,1)   
    fig.canvas.mpl_connect('button_press_event', onclick) #append event to figure
    
    xticks = np.arange(len(current_signal_chunk)) + (chunk_number * SIGNAL_CHUNK_SIZE_SAMPLES)
    ax.plot(xticks, current_signal_chunk, markersize=0.1) #plot, only hours
    
    ax.tick_params(axis='x', rotation=90) #rotate the xlabels
     
    if np.max(current_signal_chunk) < 1:
           ax.set_ylim([-6,6])
 
    plt.title(str(start_timestamp_chunk) + " - " + str(end_timestamp_chunk))
    
    plt.show()

<div class="alert alert-danger">
    <h3>Made an error?</h3>
    <p>Uncomment and execute the following cell to delete the last click.</p>
</div>

In [ ]:
# ONLY USE TO DELETE WRONG DATA - uncomment if needed
#event_dictionary = delete_last_click(event_dictionary)

<div class="alert alert-info">
    <h3>Save the labels</h3>
</div>

In [ ]:
save_path = os.path.join(day_save_path, str(str(current_CREAM_day.day_date) + "_hour-" + str(current_hour_index) + ".csv"))
event_df = pd.DataFrame(event_dictionary)
event_df["Timestamp"] = event_df["Timestamp"].apply(lambda index: start_timestamp_file + 
                                                    timedelta(seconds=(index * TIME_PER_DATAPOINT)))  
event_df.to_csv(save_path, sep=";", header=True, index=False, encoding="utf-8", decimal=".")


## 12. Hour (19:00 - 20:00)

In [ ]:
%matplotlib notebook
# DO NOT DELETE, needs to be loaded twice!
%matplotlib notebook  

event_dictionary = {
    "Filename": [],
    "Timestamp": [],
    "Amplitude": []
}

current_hour_index = 12
current_day_path = os.path.join(day_path, files_of_day[current_hour_index])
voltage, current = current_CREAM_day.load_file(current_CREAM_day.files[current_hour_index], return_noise=False)
start_timestamp_file = current_CREAM_day.files_metadata_df.iloc[current_hour_index].Start_timestamp
end_timestamp_file = current_CREAM_day.files_metadata_df.iloc[current_hour_index].End_timestamp

whole_current_signal = current  # get the current signal of the whole file
number_of_chunks = int(len(whole_current_signal) / SIGNAL_CHUNK_SIZE_SAMPLES)
all_current_signal_chunks = np.array_split(whole_current_signal,number_of_chunks)

# plot 2 minute chunks
for chunk_number, current_signal_chunk in enumerate(all_current_signal_chunks):
    start_timestamp_chunk = start_timestamp_file + timedelta(minutes=int(SIGNAL_CHUNK_SIZE_MINUTES * chunk_number)) #compute the start of the chunk
    end_timestamp_chunk = start_timestamp_chunk + timedelta(minutes=SIGNAL_CHUNK_SIZE_MINUTES)
    
    fig, ax = plt.subplots(1,1)   
    fig.canvas.mpl_connect('button_press_event', onclick) #append event to figure
    
    xticks = np.arange(len(current_signal_chunk)) + (chunk_number * SIGNAL_CHUNK_SIZE_SAMPLES)
    ax.plot(xticks, current_signal_chunk, markersize=0.1) #plot, only hours
    
    ax.tick_params(axis='x', rotation=90) #rotate the xlabels
     
    if np.max(current_signal_chunk) < 1:
           ax.set_ylim([-6,6])
 
    plt.title(str(start_timestamp_chunk) + " - " + str(end_timestamp_chunk))
    
    plt.show()

<div class="alert alert-danger">
    <h3>Made an error?</h3>
    <p>Uncomment and execute the following cell to delete the last click.</p>
</div>

In [ ]:
# ONLY USE TO DELETE WRONG DATA - uncomment if needed
#event_dictionary = delete_last_click(event_dictionary)

<div class="alert alert-info">
    <h3>Save the labels</h3>
</div>

In [ ]:
save_path = os.path.join(day_save_path, str(str(current_CREAM_day.day_date) + "_hour-" + str(current_hour_index) + ".csv"))
event_df = pd.DataFrame(event_dictionary)
event_df["Timestamp"] = event_df["Timestamp"].apply(lambda index: start_timestamp_file + 
                                                    timedelta(seconds=(index * TIME_PER_DATAPOINT)))  
event_df.to_csv(save_path, sep=";", header=True, index=False, encoding="utf-8", decimal=".")


## 13. Hour (19:00 - 20:00)

In [ ]:
%matplotlib notebook
# DO NOT DELETE, needs to be loaded twice!
%matplotlib notebook  

event_dictionary = {
    "Filename": [],
    "Timestamp": [],
    "Amplitude": []
}

current_hour_index = 13
current_day_path = os.path.join(day_path, files_of_day[current_hour_index])
voltage, current = current_CREAM_day.load_file(current_CREAM_day.files[current_hour_index], return_noise=False)
start_timestamp_file = current_CREAM_day.files_metadata_df.iloc[current_hour_index].Start_timestamp
end_timestamp_file = current_CREAM_day.files_metadata_df.iloc[current_hour_index].End_timestamp

whole_current_signal = current  # get the current signal of the whole file
number_of_chunks = int(len(whole_current_signal) / SIGNAL_CHUNK_SIZE_SAMPLES)
all_current_signal_chunks = np.array_split(whole_current_signal,number_of_chunks)

# plot 2 minute chunks
for chunk_number, current_signal_chunk in enumerate(all_current_signal_chunks):
    start_timestamp_chunk = start_timestamp_file + timedelta(minutes=int(SIGNAL_CHUNK_SIZE_MINUTES * chunk_number)) #compute the start of the chunk
    end_timestamp_chunk = start_timestamp_chunk + timedelta(minutes=SIGNAL_CHUNK_SIZE_MINUTES)
    
    fig, ax = plt.subplots(1,1)   
    fig.canvas.mpl_connect('button_press_event', onclick) #append event to figure
    
    xticks = np.arange(len(current_signal_chunk)) + (chunk_number * SIGNAL_CHUNK_SIZE_SAMPLES)
    ax.plot(xticks, current_signal_chunk, markersize=0.1) #plot, only hours
    
    ax.tick_params(axis='x', rotation=90) #rotate the xlabels
     
    if np.max(current_signal_chunk) < 1:
           ax.set_ylim([-6,6])
 
    plt.title(str(start_timestamp_chunk) + " - " + str(end_timestamp_chunk))
    
    plt.show()

<div class="alert alert-danger">
    <h3>Made an error?</h3>
    <p>Uncomment and execute the following cell to delete the last click.</p>
</div>

In [ ]:
# ONLY USE TO DELETE WRONG DATA - uncomment if needed
#event_dictionary = delete_last_click(event_dictionary)

<div class="alert alert-info">
    <h3>Save the labels</h3>
</div>

In [ ]:
save_path = os.path.join(day_save_path, str(str(current_CREAM_day.day_date) + "_hour-" + str(current_hour_index) + ".csv"))
event_df = pd.DataFrame(event_dictionary)
event_df["Timestamp"] = event_df["Timestamp"].apply(lambda index: start_timestamp_file + 
                                                    timedelta(seconds=(index * TIME_PER_DATAPOINT)))  
event_df.to_csv(save_path, sep=";", header=True, index=False, encoding="utf-8", decimal=".")


## 14. Hour (20:00 - 21:00)

In [ ]:
%matplotlib notebook
# DO NOT DELETE, needs to be loaded twice!
%matplotlib notebook  

event_dictionary = {
    "Filename": [],
    "Timestamp": [],
    "Amplitude": []
}

current_hour_index = 14
current_day_path = os.path.join(day_path, files_of_day[current_hour_index])
voltage, current = current_CREAM_day.load_file(current_CREAM_day.files[current_hour_index], return_noise=False)
start_timestamp_file = current_CREAM_day.files_metadata_df.iloc[current_hour_index].Start_timestamp
end_timestamp_file = current_CREAM_day.files_metadata_df.iloc[current_hour_index].End_timestamp

whole_current_signal = current # get the current signal of the whole file
number_of_chunks = int(len(whole_current_signal) / SIGNAL_CHUNK_SIZE_SAMPLES)
all_current_signal_chunks = np.array_split(whole_current_signal,number_of_chunks)

# plot 2 minute chunks
for chunk_number, current_signal_chunk in enumerate(all_current_signal_chunks):
    start_timestamp_chunk = start_timestamp_file + timedelta(minutes=int(SIGNAL_CHUNK_SIZE_MINUTES * chunk_number)) #compute the start of the chunk
    end_timestamp_chunk = start_timestamp_chunk + timedelta(minutes=SIGNAL_CHUNK_SIZE_MINUTES)
    
    fig, ax = plt.subplots(1,1)   
    fig.canvas.mpl_connect('button_press_event', onclick) #append event to figure
    
    xticks = np.arange(len(current_signal_chunk)) + (chunk_number * SIGNAL_CHUNK_SIZE_SAMPLES)
    ax.plot(xticks, current_signal_chunk, markersize=0.1) #plot, only hours
    
    ax.tick_params(axis='x', rotation=90) #rotate the xlabels
     
    if np.max(current_signal_chunk) < 1:
           ax.set_ylim([-6,6])
 
    plt.title(str(start_timestamp_chunk) + " - " + str(end_timestamp_chunk))
    
    plt.show()

<div class="alert alert-danger">
    <h3>Made an error?</h3>
    <p>Uncomment and execute the following cell to delete the last click.</p>
</div>

In [ ]:
# ONLY USE TO DELETE WRONG DATA - uncomment if needed
#event_dictionary = delete_last_click(event_dictionary)

<div class="alert alert-info">
    <h3>Save the labels</h3>
</div>

In [ ]:
save_path = os.path.join(day_save_path, str(str(current_CREAM_day.day_date) + "_hour-" + str(current_hour_index) + ".csv"))
event_df = pd.DataFrame(event_dictionary)
event_df["Timestamp"] = event_df["Timestamp"].apply(lambda index: start_timestamp_file + 
                                                    timedelta(seconds=(index * TIME_PER_DATAPOINT)))  

event_df.to_csv(save_path, sep=";", header=True, index=False, encoding="utf-8", decimal=".")


## 15. Hour (21:00 - 22:00)

In [ ]:
%matplotlib notebook
# DO NOT DELETE, needs to be loaded twice!
%matplotlib notebook  

event_dictionary = {
    "Filename": [],
    "Timestamp": [],
    "Amplitude": []
}

current_hour_index = 15
current_day_path = os.path.join(day_path, files_of_day[current_hour_index])
voltage, current = current_CREAM_day.load_file(current_CREAM_day.files[current_hour_index], return_noise=False)
start_timestamp_file = current_CREAM_day.files_metadata_df.iloc[current_hour_index].Start_timestamp
end_timestamp_file = current_CREAM_day.files_metadata_df.iloc[current_hour_index].End_timestamp

whole_current_signal = current  # get the current signal of the whole file
number_of_chunks = int(len(whole_current_signal) / SIGNAL_CHUNK_SIZE_SAMPLES)
all_current_signal_chunks = np.array_split(whole_current_signal,number_of_chunks)

# plot 2 minute chunks
for chunk_number, current_signal_chunk in enumerate(all_current_signal_chunks):
    start_timestamp_chunk = start_timestamp_file + timedelta(minutes=int(SIGNAL_CHUNK_SIZE_MINUTES * chunk_number)) #compute the start of the chunk
    end_timestamp_chunk = start_timestamp_chunk + timedelta(minutes=SIGNAL_CHUNK_SIZE_MINUTES)
    
    fig, ax = plt.subplots(1,1)   
    fig.canvas.mpl_connect('button_press_event', onclick) #append event to figure
    
    xticks = np.arange(len(current_signal_chunk)) + (chunk_number * SIGNAL_CHUNK_SIZE_SAMPLES)
    ax.plot(xticks, current_signal_chunk, markersize=0.1) #plot, only hours
    
    ax.tick_params(axis='x', rotation=90) #rotate the xlabels
     
    if np.max(current_signal_chunk) < 1:
           ax.set_ylim([-6,6])
 
    plt.title(str(start_timestamp_chunk) + " - " + str(end_timestamp_chunk))
    
    plt.show()

<div class="alert alert-danger">
    <h3>Made an error?</h3>
    <p>Uncomment and execute the following cell to delete the last click.</p>
</div>

In [ ]:
# ONLY USE TO DELETE WRONG DATA - uncomment if needed
#event_dictionary = delete_last_click(event_dictionary)

<div class="alert alert-info">
    <h3>Save the labels</h3>
</div>

In [ ]:
save_path = os.path.join(day_save_path, str(str(current_CREAM_day.day_date) + "_hour-" + str(current_hour_index) + ".csv"))
event_df = pd.DataFrame(event_dictionary)
event_df["Timestamp"] = event_df["Timestamp"].apply(lambda index: start_timestamp_file + 
                                               timedelta(seconds=(index * TIME_PER_DATAPOINT)))  
event_df.to_csv(save_path, sep=";", header=True, index=False, encoding="utf-8", decimal=".")

# End of the day

<div class="alert alert-info">
    <h3>Are all hours/chunks of the day labeled? Did you save the data?</h3>
    <p>Please jump to the top of the notebook and select the next day, by changing the <b>CURRENT_DAY</b> variable.
</div>

# End of the labeling procedure (i.e. all days done)

<div class="alert alert-danger">
    <h1> Merge the single day files if you have labeled everything </h1>
    <p> Uncomment the following cell, to load the individual files (the per hour file created before in the labeling process) and merge them into a final file. 
        This final file is then save at the data location. The file is uncommented to prevent from accidentially running this code.</p>
    <p> <b> The code also creates the "Event_Type" column, indicating "On" or "Off" events by using the amplitude information from the labeling </b> </p>
</div>

In [ ]:
"""ALL_DAYS = ["2018-08-23" , "2018-08-24" , "2018-08-25",  "2018-08-26" , "2018-08-27" , "2018-08-28" ,
"2018-08-29", "2018-08-30", "2018-08-31", "2018-09-01", "2018-09-02" , "2018-09-03" ,  "2018-09-04" ,
"2018-09-05", "2018-09-06", "2018-09-07", "2018-09-08" , "2018-09-09" , "2018-09-10", "2018-09-11", "2018-09-12" 
"2018-09-13" ,"2018-09-14" ,"2018-09-15" ,  "2018-09-16", "2018-09-17", "2018-09-18","2018-09-19"  , "2018-09-20" ,
"2018-09-21" , "2018-09-22" ,  "2018-09-23" ,"2018-09-24" ,"2018-09-25" ,"2018-09-26" , "2018-09-27", "2018-09-28" ,
"2018-09-29" , "2018-09-30" , "2018-10-01" ,"2018-10-02" , "2018-10-03" ,"2018-10-04", "2018-10-05" , "2018-10-06" ,
"2018-10-07", "2018-10-08"]

PATH_TO_SAVE_LOCATION = os.path.abspath(os.path.join("..", "..", "Datasets", "CREAM",  ))

# Load all the individually labeled files for all days
all_labeled_files = []
for day in ALL_DAYS:
    day_path = os.path.join(PATH_TO_SAVE_LOCATION, day)
    labeled_files = glob.glob(os.path.join(day_path, "*.csv"))
    all_labeled_files.extend(labeled_files)

all_labeled_dfs = []
import pdb
for f in all_labeled_files:
    print(os.path.basename(f))
        
    event_df = pd.read_csv(f, sep=";", encoding="utf-8", decimal=".")
    all_labeled_dfs.append(event_df)
    
pdb.set_trace()
component_df = pd.concat(all_labeled_dfs)

timezone = current_CREAM_day.get_datetime_from_filepath(current_CREAM_day.files[0]).tzinfo #get the correct timzone
component_df.Timestamp = component_df.Timestamp.apply(pd.Timestamp)
component_df.Timestamp = component_df.Timestamp.dt.tz_localize(timezone)
component_df.sort_values("Timestamp", inplace=True)

pdb.set_trace()
# Add the "Event_Type" column and fill it
# This is done by computing the differences between consecutive rows and the following rows.
# We need to discard the last event since its event type cannot be computed due to the lack of a follow up event.
# Since it's only a relatively small number, we discard them to ensure the data integrity.

component_df.Amplitude = component_df.Amplitude.apply(pd.to_numeric)

diff = np.diff(component_df.Amplitude.values)

component_df = component_df.iloc[0:-1]  # discard the last event, as explained above
component_df['Event_Type'] = diff

component_df = component_df[component_df.Event_Type != 0]  # discard 0 diffs

component_df.Event_Type = component_df.Event_Type.apply(lambda x: "On" if x > 0 else "Off")
pdb.set_trace()
# Adjust the timestamp, to make them match the product and maintenance event timestamps with respect to the timezone offset


component_df.to_csv(os.path.join(PATH_TO_DATA, "raw_coffee_maker_logs", "raw_component_events.csv"), index=False)
"""